In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update 
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-securi

In [2]:
# Start a spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Vine_Review_Analysis").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [4]:
# Filter the data to show reviews with 20 or more total_votes. 
reviews_20_votes_or_more_df = df.filter("total_votes >= 20")
reviews_20_votes_or_more_df.show(50)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    7142190| R4PKAZRQJJX14|B00QZLVCU0|     210935604|Valve - DOTA 2 St...|     Video Games|          1|           21|         34|   N|                N|What store doesn'...|Who pays 4 dollar...| 2015-08-31|
|         US|    1085641|R2CI0Y288CC7E2|B00RHI62GY|     626589765|ONE PIECE Pirate ...|     Video Games|          1|    

In [5]:
helpful_reviews_20_votes_df = reviews_20_votes_or_more_df.filter("helpful_votes/total_votes >= .5")
helpful_reviews_20_votes_df.show(50)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    7142190| R4PKAZRQJJX14|B00QZLVCU0|     210935604|Valve - DOTA 2 St...|     Video Games|          1|           21|         34|   N|                N|What store doesn'...|Who pays 4 dollar...| 2015-08-31|
|         US|    1085641|R2CI0Y288CC7E2|B00RHI62GY|     626589765|ONE PIECE Pirate ...|     Video Games|          1|    

In [6]:
from pyspark.sql.functions import col

paid_reviews_df = helpful_reviews_20_votes_df.filter(col("vine") == "Y")
paid_reviews_df.show(20)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48771843|R3KKUSGFZWSUIY|B00YXO5UXG|     555176890|Turtle Beach - Ea...|     Video Games|          5|           56|         63|   Y|                N|Quality Chat Headset|Not every situati...| 2015-08-04|
|         US|   53080186|R10FO5UKKVZBK2|B00XO041RQ|     238654494|PDP AG7 True Wire...|     Video Games|          3|    

In [7]:
unpaid_reviews_df = helpful_reviews_20_votes_df.filter(col("vine") == "N")
unpaid_reviews_df.show(20)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    7142190| R4PKAZRQJJX14|B00QZLVCU0|     210935604|Valve - DOTA 2 St...|     Video Games|          1|           21|         34|   N|                N|What store doesn'...|Who pays 4 dollar...| 2015-08-31|
|         US|    1085641|R2CI0Y288CC7E2|B00RHI62GY|     626589765|ONE PIECE Pirate ...|     Video Games|          1|    

In [8]:
total_reviews = helpful_reviews_20_votes_df.select("review_id").count()
total_reviews

40565

In [9]:
five_star_reviews = helpful_reviews_20_votes_df.filter("star_rating == 5").count()
five_star_reviews

15711

In [10]:
paid_five_star_reviews = paid_reviews_df.filter("star_rating == 5").count()
paid_five_star_reviews

48

In [11]:
unpaid_five_star_reviews = unpaid_reviews_df.filter("star_rating == 5").count()
unpaid_five_star_reviews

15663

In [12]:
count_paid_reviews = paid_reviews_df.select("review_id").count()
count_paid_reviews

94